In [ ]:
import pandas as pd
import numpy as np

#Function


def calculate_numerical_stats(series):
    return {
        "Mean ± SD": f"{series.mean():.1f}±{series.std():.1f}",
        "Median (Q1, Q3)": f"{series.median():.1f} ({series.quantile(0.25):.1f},{series.quantile(0.75):.1f})",
        "(Min, Max)": f"({series.min():.1f},{series.max():.1f})"
    }
# Function to calculate statistics for categorical columns
def calculate_categorical_stats(series):
    total_count = len(series)
    counts = series.value_counts()
    return {val: f"{(count / total_count):.1%} ({count}/{total_count})" for val, count in counts.items()}



# Function to calculate the percentage and counts for categorical columns with Yes/No responses
def calculate_percentage_and_counts(df, column_name):
    counts = df[column_name].value_counts()
    total = len(df)
    percentages = {
        'Yes': f"{(counts.get('Y', 0)/total)*100:.1f}% ({counts.get('Y', 0)}/{total})",
        'No': f"{(counts.get('N', 0)/total)*100:.1f}% ({counts.get('N', 0)}/{total})"
    }
    return percentages

#Input Files


#registry New Files

DM = pd.read_csv("/Users/pankajkumartiwari/Downloads/Lucid_july/Registry/DM.csv")
BE = pd.read_csv("/Users/pankajkumartiwari/Downloads/Lucid_july/Registry/BE.csv")





#Chohort1  ##ALL

df1 = DM[["SUBJID","SEX","AGE","RACE_BLACK","RACE_WHITE_NH","RACE_WHITE_H","RACE_HNPI","RACE_AIAN","RACE_ASIAN" ]]
df2 = BE[["SUBJID","BEGERD","BMI","BESMOKE","BEFAM","BEEXPO","BEACIDCM", "BEGERDWK", "BESMOKESTS","BEGERDYRS","BEOB"]]


#df1:

df1.loc[df1['AGE'] < 50, 'AGE_1'] = 0
df1.loc[df1['AGE'] >= 50, 'AGE_1'] = 1


df1.loc[df1['AGE'] <= 30, 'AGE_2'] = 1
df1.loc[df1['AGE'] > 30, 'AGE_2'] = 0


df1.loc[df1['SEX'] == "F" ,'SEX_1'] = 0
df1.loc[df1['SEX'] == "M", 'SEX_1'] = 1


#df1.loc[df1['RACE_WHITE_NH'] == "Y", 'RACE_WHITE_NH_1'] = 1
#df1.loc[df1['RACE_WHITE_NH'] == "N", 'RACE_WHITE_NH_1'] = 0

df1['RACE_WHITE_NH_1'] = ((df1['RACE_WHITE_NH'] == 'Y') &

                   (df1['RACE_WHITE_H'] != 'Y') &

                   (df1['RACE_BLACK'] != 'Y') &

                   (df1['RACE_AIAN'] != 'Y') &

                   (df1['RACE_ASIAN'] != 'Y') &

                   (df1['RACE_HNPI'] != 'Y')).astype(int)



df1.loc[df1['RACE_BLACK'] == "Y", 'RACE_BLACK_1'] = 1
df1.loc[df1['RACE_BLACK'] == "N", 'RACE_BLACK_1'] = 0


df2.loc[df2['BEGERD'] == "Y", 'BEGERD_1'] = 1
df2.loc[df2['BEGERD'] == "N", 'BEGERD_1'] = 0


df2.loc[df2['BMI'] < 29.5, 'BMI_1'] = 0
df2.loc[df2['BMI'] >= 29.5, 'BMI_1'] = 1


df2.loc[df2['BESMOKE'] == "Y", 'BESMOKE_1'] = 1
df2.loc[df2['BESMOKE'] == "N", 'BESMOKE_1'] = 0


df2.loc[df2['BEFAM'] == "Y", 'BEFAM_1'] = 1
df2.loc[df2['BEFAM'] == "N", 'BEFAM_1'] = 0


df2.loc[df2['BEEXPO'] == "Y", 'BEEXPO_1'] = 1
df2.loc[df2['BEEXPO'] == "N", 'BEEXPO_1'] = 0


merged_data = df1.merge(df2, on = 'SUBJID', how = 'inner')


merged_data['AGE_1']  = merged_data['AGE_1'].fillna(0)
merged_data['AGE_2']  = merged_data['AGE_2'].fillna(0)
merged_data['SEX_1'] = merged_data['SEX_1'].fillna(0)
merged_data['RACE_WHITE_NH_1'] = merged_data['RACE_WHITE_NH_1'].fillna(0)
merged_data['BEGERD_1'] = merged_data['BEGERD_1'].fillna(0)
merged_data['BMI_1'] = merged_data['BMI_1'].fillna(0)
merged_data['BESMOKE_1'] = merged_data['BESMOKE_1'].fillna(0)
merged_data['BEFAM_1'] = merged_data['BEFAM_1'].fillna(0)


merged_data['RF'] =  merged_data['AGE_1'] + merged_data['SEX_1']+  merged_data['RACE_WHITE_NH_1'] +merged_data['BEGERD_1']  + merged_data['BMI_1']+ merged_data['BESMOKE_1']+ merged_data['BEFAM_1']

merged_data['AGA_SUM'] = merged_data['RF'] +merged_data['BEEXPO_1']


merged_data.loc[merged_data['AGA_SUM'] >= 3, 'AGA_Flag'] = "Y"
merged_data.loc[merged_data['AGA_SUM'] < 3, 'AGA_Flag'] = "N"


merged_data.loc[merged_data['RF'] >=3, '3_or_more_established_risk_factors'] = "Y"
merged_data.loc[merged_data['RF'] < 3, '3_or_more_established_risk_factors'] = "N"


merged_data.loc[merged_data['RF'] > 0, '1_or_more_established_risk_factors'] = 1
merged_data.loc[merged_data['RF'] < 0, '1_or_more_established_risk_factors'] = 0


merged_data.loc[merged_data['RF'] >= 4, '4_or_more_established_risk_factors'] = "Y"
merged_data.loc[merged_data['RF'] < 4, '4_or_more_established_risk_factors'] = "N"


merged_data['GERD_AND_3RF'] = np.where( ( (merged_data['3_or_more_established_risk_factors'] == "Y") & (merged_data['BEGERD_1'] == 1)  ), "Y", "N")

merged_data['GERD_AND_3RF_FF'] = np.where( ( (merged_data['3_or_more_established_risk_factors'] == "Y") & (merged_data['BEGERD_1'] == 1) & (merged_data['BEEXPO_1'] == 1) ), "Y", "N")

FF = merged_data[merged_data["BEEXPO"] == "Y"]


FF['FF_2ormoreRF'] = np.where( ( (FF['BEEXPO_1'] == 1) & (FF['RF'] >=2)  ), "Y","N")

FF['FF_3ormoreRF'] = np.where( ( (FF['BEEXPO_1'] == 1) & (FF['RF'] >=3)  ), "Y", "N")

FF['FF_4ormoreRF'] = np.where( ( (FF['BEEXPO_1'] == 1) & (FF['RF'] >=4)  ), "Y", "N")

f3 = merged_data

f4 = merged_data[merged_data["3_or_more_established_risk_factors"] == "Y"]

df2_results = {
     'Age (at time of EsoCheck cell collection; years) ':  calculate_numerical_stats(f3['AGE']),
     'Biological Sex' :calculate_categorical_stats(f3['SEX']),
     'White (Caucasian) Non-Hispanic' : calculate_categorical_stats(f3['RACE_WHITE_NH_1']),
     'Black or African American' : calculate_categorical_stats(f3['RACE_BLACK']),
     'White (Caucasian) Hispanic' : calculate_categorical_stats(f3['RACE_WHITE_H']),
     'American Indian or Alaskan Native': calculate_categorical_stats(f3['RACE_AIAN']),
     'Native Hawaiian or other Pacific Islander' : calculate_categorical_stats(f3['RACE_HNPI']),
     'Asian (Asian Indian; Chinese; Filipino; Japanese; Korean; Vietnamese; Other Asian)': calculate_categorical_stats(f3['RACE_ASIAN']),
     'Cigarette Smoker (past or present)?':  calculate_categorical_stats(f3['BESMOKE_1']),
     'If positive history of cigarette smoking, indicate current status:' : calculate_categorical_stats(f3['BESMOKESTS']),
     'History of gastroesophageal reflux disease (GERD)?': calculate_categorical_stats(f3['BEGERD']),
     'If positive history of GERD, indicate number of years with disease' : calculate_categorical_stats(f3['BEGERDYRS']),
     'If positive history of GERD, how many times per week are symptoms experienced?'  : calculate_categorical_stats(f3['BEGERDWK']),
     'Are GERD symptoms controlled with the acid-controlling medications?'  : calculate_categorical_stats(f3['BEACIDCM']),
     'First degree family members with known diagnosis of Barretts Esophagus (BE) and/or esophageal adenocarcinoma (EAC)?': calculate_categorical_stats(f3['BEFAM']),
     'History of occupational and/or environmental exposure to agents that may increase risk of BE or EAC?'  : calculate_categorical_stats(f3['BEEXPO']),
     'Obese (defined as BMI ≥30kg/m2)' : calculate_categorical_stats(f3['BMI_1']),
     'Age >= 50' : calculate_categorical_stats(f3['AGE_1']),
     'Age <= 30' : calculate_categorical_stats(f3['AGE_2']),
     'Body mass index (BMI)' : calculate_numerical_stats(f3['BMI']),
     'Number of Risk Factors (any)' :calculate_numerical_stats(f3['1_or_more_established_risk_factors']),
     'Have: BEGERD = Yes, AND meet any 3 (or more) of the additional risk factors' : calculate_categorical_stats(f3['GERD_AND_3RF']),
     'If positive history of GERD, indicate number of years with disease': calculate_numerical_stats(f3['BEGERDYRS']),
     'If positive history of GERD, how many times per week are symptoms experienced?': calculate_numerical_stats(f3['BEGERDWK']),
     'Are GERD symptoms controlled with the acid-controlling medications?': calculate_categorical_stats(f3['BEACIDCM']),
     '3 or more established risk factors' : calculate_categorical_stats(f3['3_or_more_established_risk_factors']),
     'AGA (+) Cohort (3 or more risk factors, including occupational/environmental exposure as a presumed EAC risk factor)' : calculate_categorical_stats(f3['AGA_Flag']),
     '4 or more established risk factors ' : calculate_categorical_stats(f3['4_or_more_established_risk_factors']),
     'Firefighter (FF) cohort': calculate_categorical_stats(FF['BEEXPO']),
     'Firefighter (FF) cohort with 2 or more risk factors' :calculate_categorical_stats(FF['FF_2ormoreRF']),
     'Firefighter (FF) cohort with 3 or more risk factors' :calculate_categorical_stats(FF['FF_3ormoreRF']),
     'Firefighter (FF) cohort with 4 or more risk factors' :calculate_categorical_stats(FF['FF_4ormoreRF']),
     'Firefighter (FF) cohort with GERD and 3 or more risk factors' :calculate_categorical_stats(FF['GERD_AND_3RF_FF'])
     }
df3_results = {

  'Age (at time of EsoCheck cell collection; years) ':  calculate_numerical_stats(f4['AGE']),
  'Biological Sex' :calculate_categorical_stats(f4['SEX']),
  'White (Caucasian) Non-Hispanic' : calculate_categorical_stats(f4['RACE_WHITE_NH_1']),
  'Black or African American' : calculate_categorical_stats(f4['RACE_BLACK']),
  'White (Caucasian) Hispanic' : calculate_categorical_stats(f4['RACE_WHITE_H']),
  'American Indian or Alaskan Native': calculate_categorical_stats(f4['RACE_AIAN']),
  'Native Hawaiian or other Pacific Islander' : calculate_categorical_stats(f4['RACE_HNPI']),
  'Asian (Asian Indian; Chinese; Filipino; Japanese; Korean; Vietnamese; Other Asian)': calculate_categorical_stats(f4['RACE_ASIAN']),
  'Cigarette Smoker (past or present)?':  calculate_categorical_stats(f4['BESMOKE_1']),
  'If positive history of cigarette smoking, indicate current status:' : calculate_categorical_stats(f4['BESMOKESTS']),
  'History of gastroesophageal reflux disease (GERD)?': calculate_categorical_stats(f4['BEGERD']),
  'If positive history of GERD, indicate number of years with disease' : calculate_categorical_stats(f4['BEGERDYRS']),
  'If positive history of GERD, how many times per week are symptoms experienced?'  : calculate_categorical_stats(f4['BEGERDWK']),
  'Are GERD symptoms controlled with the acid-controlling medications?'  : calculate_categorical_stats(f4['BEACIDCM']),
  'First degree family members with known diagnosis of Barretts Esophagus (BE) and/or esophageal adenocarcinoma (EAC)?': calculate_categorical_stats(f4['BEFAM']),
  'History of occupational and/or environmental exposure to agents that may increase risk of BE or EAC?'  : calculate_categorical_stats(f4['BEEXPO']),
  'Obese (defined as BMI ≥30kg/m2)' : calculate_categorical_stats(f4['BMI_1']),
  'Age >= 50' : calculate_categorical_stats(f4['AGE_1']),
  'Age <= 30' : calculate_categorical_stats(f4['AGE_2']),
  'Body mass index (BMI)' : calculate_numerical_stats(f4['BMI']),
  'Number of Risk Factors (any)' :calculate_numerical_stats(f4['1_or_more_established_risk_factors']),
  'Have: BEGERD = Yes, AND meet any 3 (or more) of the additional risk factors' : calculate_categorical_stats(f4['GERD_AND_3RF']),
  'If positive history of GERD, indicate number of years with disease': calculate_numerical_stats(f4['BEGERDYRS']),
  'If positive history of GERD, how many times per week are symptoms experienced?': calculate_numerical_stats(f4['BEGERDWK']),
  'Are GERD symptoms controlled with the acid-controlling medications?': calculate_categorical_stats(f4['BEACIDCM']),
  '4 or more established risk factors ' : calculate_categorical_stats(f4['4_or_more_established_risk_factors'])
}


ff = pd.DataFrame.from_dict(df2_results)
pp = ff.transpose()
pp.to_csv('registry_1d_july.csv')


ff1 = pd.DataFrame.from_dict(df3_results)
pp1 = ff1.transpose()
pp1.to_csv('registry_2d_july.csv')


merged_data.to_csv('registry_july_validation.csv')